# Using OpenVINO Docker containers for development and deployment


## Introduction

The purpose of this tutorial is to examine a sample application that was created using the [Intel® Distribution of OpenVINO™ toolkit](https://software.intel.com/openvino-toolkit).  

This tutorial will go step-by-step to demonstrate [Speech Recognition DeepSpeech demo](https://docs.openvino.ai/latest/omz_demos_speech_recognition_deepspeech_demo_python.html) using OpenVINO™ Docker container. Recognition is performed using a pre-trained network - Mozilla* DeepSpeech. We will also cover model download and conversion via Model Optimizer. The model is capable of recognizing the speech in English.  Inference will be executed using the same CPU(s) running this Jupyter* Notebook.


## Prerequisites

This tutorial requires the following:
- Unix* host
- Installed and run [Docker](https://docs.docker.com/engine/install/)* engine/service on the host


## Preparation

OpenVINO has a couple of dependencies which need to be present on your computer. Additionally, to install some of them, you need to have root/admin rights. This might not be desirable. Using Docker represents much cleaner way. Especially when there is an image prepared for you on [Docker Hub OpenVINO organization](https://hub.docker.com/r/openvino).

We have some types of Docker images depend on distribution:

*   **runtime** - contains OpenVINO Runtime (Inference Engine core, nGraph) libs for each supported device (CPU, GNA, GPU, VPU)
*   **dev** - contains runtime part plus samples and Python development tools: Model Optimizer, Post training Optimization tool (POT), Accuracy checker, Open Model Zoo tools (downloader, converter)

You can search OpenVINO Docker images on Docker Hub or via cli:





```
docker search --format "{{.Name}}: {{.Description}}" --no-trunc --limit 100  openvino | grep ^openvino/ | grep -v DEPRECATED
```




Output will be like this:
```
openvino/workbench: OpenVINO™ DL Workbench is a web GUI to simplify DL models inference and tuning on Intel® devices.
openvino/ubuntu18_runtime: Intel® Distribution of OpenVINO™ toolkit Docker image for Ubuntu* 18.04 LTS
openvino/ubuntu18_dev: Intel® Distribution of OpenVINO™ toolkit Docker image for Ubuntu* 18.04 LTS
openvino/ubuntu20_runtime: Intel® Distribution of OpenVINO™ toolkit Docker image for Ubuntu* 20.04 LTS
openvino/model_server: Intel® Distribution of OpenVINO™ Model Server Docker images
openvino/cvat_ui: Computer Vision Annotation Tool (CVAT) frontend UI image
openvino/cvat_server: Computer Vision Annotation Tool (CVAT) annotation backend image
openvino/winserver2019_runtime: Intel® Distribution of OpenVINO™ toolkit Docker image for Windows Server Core base LTSC 2019
openvino/ubuntu20_dev: Intel® Distribution of OpenVINO™ toolkit Docker image for Ubuntu* 20.04 LTS
```



## Part 1. Using OpenVINO development Docker container for model conversion and execution.

### 1.	Pull dev OpenVINO docker image and run it to download a model from the OpenVINO Model Zoo.


The container can be used to run all the included tools without the need to install them on the host. Here we will use the tool `omz_downloader`. The command below can list all the models available in the OpenVINO Model Zoo:

In [1]:
!docker run openvino/ubuntu20_dev:2022.2.0 omz_downloader --print_all

Sphereface
aclnet
aclnet-int8
action-recognition-0001
age-gender-recognition-retail-0013
alexnet
anti-spoof-mn3
asl-recognition-0004
background-matting-mobilenetv2
bert-base-ner
bert-large-uncased-whole-word-masking-squad-0001
bert-large-uncased-whole-word-masking-squad-emb-0001
bert-large-uncased-whole-word-masking-squad-int8-0001
bert-small-uncased-whole-word-masking-squad-0001
bert-small-uncased-whole-word-masking-squad-0002
bert-small-uncased-whole-word-masking-squad-emb-int8-0001
bert-small-uncased-whole-word-masking-squad-int8-0002
brain-tumor-segmentation-0001
brain-tumor-segmentation-0002
caffenet
cocosnet
colorization-siggraph
colorization-v2
common-sign-language-0001
common-sign-language-0002
convnext-tiny
ctdet_coco_dlav0_512
ctpn
deblurgan-v2
deeplabv3
densenet-121
densenet-121-tf
detr-resnet50
dla-34
driver-action-recognition-adas-0002
drn-d-38
efficientdet-d0-tf
efficientdet-d1-tf
efficientnet-b0
efficientnet-b0-pytorch
efficientnet-v2-b0
efficientnet-v2-s
emotions-recogn

`docker run` command below will execute `omz_downloader` script to download `mozilla-deepspeech-0.6.1` model to `model` folder on the host.

* `-u $(id -u)` this parameter start to container in the current user security context. It ensures the container will have permissions in the mounted folder to save the results and keep the permissions to those saved content.
* `--rm` option needs to remove container after execution.
* `-v $(pwd)/model:/tmp/model` option needs to mount host folder model from the working directory to the container.


In [2]:
!mkdir model
!docker run -u $(id -u) --rm -v $(pwd)/model:/tmp/model openvino/ubuntu20_dev:2022.2.0 omz_downloader --name mozilla-deepspeech-0.6.1 -o /tmp/model

################|| Downloading mozilla-deepspeech-0.6.1 ||################

========== Downloading /tmp/model/public/mozilla-deepspeech-0.6.1/deepspeech-0.6.1-models.tar.gz
... 0%, 1024 KB, 5030 KB/s, 0 seconds passed
... 0%, 2048 KB, 7038 KB/s, 0 seconds passed
... 0%, 3072 KB, 8140 KB/s, 0 seconds passed
... 0%, 4096 KB, 9380 KB/s, 0 seconds passed
... 0%, 5120 KB, 10131 KB/s, 0 seconds passed
... 0%, 6144 KB, 10923 KB/s, 0 seconds passed
... 0%, 7168 KB, 11694 KB/s, 0 seconds passed
... 0%, 8192 KB, 12391 KB/s, 0 seconds passed
... 0%, 9216 KB, 12886 KB/s, 0 seconds passed
... 0%, 10240 KB, 13430 KB/s, 0 seconds passed
... 0%, 11264 KB, 13928 KB/s, 0 seconds passed
... 1%, 12288 KB, 14462 KB/s, 0 seconds passed
... 1%, 13312 KB, 14911 KB/s, 0 seconds passed
... 1%, 14336 KB, 15367 KB/s, 0 seconds passed
... 1%, 15360 KB, 15820 KB/s, 0 seconds passed
... 1%, 16384 KB, 16252 KB/s, 1 seconds passed
... 1%, 17408 KB, 16634 KB/s, 1 seconds passed
... 1%, 18432 KB, 17102 KB/s, 1 seconds p

... 14%, 174080 KB, 34843 KB/s, 4 seconds passed
... 14%, 175104 KB, 34979 KB/s, 5 seconds passed
... 14%, 176128 KB, 35124 KB/s, 5 seconds passed
... 14%, 177152 KB, 35156 KB/s, 5 seconds passed
... 14%, 178176 KB, 35292 KB/s, 5 seconds passed
... 14%, 179200 KB, 35237 KB/s, 5 seconds passed
... 15%, 180224 KB, 35350 KB/s, 5 seconds passed
... 15%, 181248 KB, 35377 KB/s, 5 seconds passed
... 15%, 182272 KB, 35500 KB/s, 5 seconds passed
... 15%, 183296 KB, 35577 KB/s, 5 seconds passed
... 15%, 184320 KB, 34781 KB/s, 5 seconds passed
... 15%, 185344 KB, 34724 KB/s, 5 seconds passed
... 15%, 186368 KB, 34842 KB/s, 5 seconds passed
... 15%, 187392 KB, 34771 KB/s, 5 seconds passed
... 15%, 188416 KB, 34897 KB/s, 5 seconds passed
... 15%, 189440 KB, 34950 KB/s, 5 seconds passed
... 15%, 190464 KB, 35073 KB/s, 5 seconds passed
... 15%, 191488 KB, 35123 KB/s, 5 seconds passed
... 16%, 192512 KB, 35232 KB/s, 5 seconds passed
... 16%, 193536 KB, 35313 KB/s, 5 seconds passed
... 16%, 194560 KB, 

... 28%, 346112 KB, 36258 KB/s, 9 seconds passed
... 28%, 347136 KB, 36305 KB/s, 9 seconds passed
... 29%, 348160 KB, 36113 KB/s, 9 seconds passed
... 29%, 349184 KB, 36180 KB/s, 9 seconds passed
... 29%, 350208 KB, 36252 KB/s, 9 seconds passed
... 29%, 351232 KB, 36246 KB/s, 9 seconds passed
... 29%, 352256 KB, 36286 KB/s, 9 seconds passed
... 29%, 353280 KB, 36201 KB/s, 9 seconds passed
... 29%, 354304 KB, 36162 KB/s, 9 seconds passed
... 29%, 355328 KB, 36226 KB/s, 9 seconds passed
... 29%, 356352 KB, 36182 KB/s, 9 seconds passed
... 29%, 357376 KB, 36250 KB/s, 9 seconds passed
... 29%, 358400 KB, 36149 KB/s, 9 seconds passed
... 29%, 359424 KB, 36214 KB/s, 9 seconds passed
... 30%, 360448 KB, 36275 KB/s, 9 seconds passed
... 30%, 361472 KB, 36251 KB/s, 9 seconds passed
... 30%, 362496 KB, 36266 KB/s, 9 seconds passed
... 30%, 363520 KB, 36232 KB/s, 10 seconds passed
... 30%, 364544 KB, 36209 KB/s, 10 seconds passed
... 30%, 365568 KB, 36275 KB/s, 10 seconds passed
... 30%, 366592 K

... 42%, 515072 KB, 36227 KB/s, 14 seconds passed
... 43%, 516096 KB, 36265 KB/s, 14 seconds passed
... 43%, 517120 KB, 36033 KB/s, 14 seconds passed
... 43%, 518144 KB, 36009 KB/s, 14 seconds passed
... 43%, 519168 KB, 36056 KB/s, 14 seconds passed
... 43%, 520192 KB, 36023 KB/s, 14 seconds passed
... 43%, 521216 KB, 36070 KB/s, 14 seconds passed
... 43%, 522240 KB, 35869 KB/s, 14 seconds passed
... 43%, 523264 KB, 35914 KB/s, 14 seconds passed
... 43%, 524288 KB, 35966 KB/s, 14 seconds passed
... 43%, 525312 KB, 35930 KB/s, 14 seconds passed
... 43%, 526336 KB, 35976 KB/s, 14 seconds passed
... 43%, 527360 KB, 35663 KB/s, 14 seconds passed
... 44%, 528384 KB, 35708 KB/s, 14 seconds passed
... 44%, 529408 KB, 35756 KB/s, 14 seconds passed
... 44%, 530432 KB, 35727 KB/s, 14 seconds passed
... 44%, 531456 KB, 35773 KB/s, 14 seconds passed
... 44%, 532480 KB, 35727 KB/s, 14 seconds passed
... 44%, 533504 KB, 35772 KB/s, 14 seconds passed
... 44%, 534528 KB, 35821 KB/s, 14 seconds passed


... 56%, 684032 KB, 35259 KB/s, 19 seconds passed
... 57%, 685056 KB, 35294 KB/s, 19 seconds passed
... 57%, 686080 KB, 35310 KB/s, 19 seconds passed
... 57%, 687104 KB, 35341 KB/s, 19 seconds passed
... 57%, 688128 KB, 35376 KB/s, 19 seconds passed
... 57%, 689152 KB, 35396 KB/s, 19 seconds passed
... 57%, 690176 KB, 35428 KB/s, 19 seconds passed
... 57%, 691200 KB, 35266 KB/s, 19 seconds passed
... 57%, 692224 KB, 35300 KB/s, 19 seconds passed
... 57%, 693248 KB, 35338 KB/s, 19 seconds passed
... 57%, 694272 KB, 35315 KB/s, 19 seconds passed
... 57%, 695296 KB, 35349 KB/s, 19 seconds passed
... 58%, 696320 KB, 35347 KB/s, 19 seconds passed
... 58%, 697344 KB, 35377 KB/s, 19 seconds passed
... 58%, 698368 KB, 35372 KB/s, 19 seconds passed
... 58%, 699392 KB, 35376 KB/s, 19 seconds passed
... 58%, 700416 KB, 35404 KB/s, 19 seconds passed
... 58%, 701440 KB, 35360 KB/s, 19 seconds passed
... 58%, 702464 KB, 35387 KB/s, 19 seconds passed
... 58%, 703488 KB, 35423 KB/s, 19 seconds passed


... 71%, 855040 KB, 35513 KB/s, 24 seconds passed
... 71%, 856064 KB, 35510 KB/s, 24 seconds passed
... 71%, 857088 KB, 35517 KB/s, 24 seconds passed
... 71%, 858112 KB, 35521 KB/s, 24 seconds passed
... 71%, 859136 KB, 35545 KB/s, 24 seconds passed
... 71%, 860160 KB, 35470 KB/s, 24 seconds passed
... 71%, 861184 KB, 35497 KB/s, 24 seconds passed
... 71%, 862208 KB, 35526 KB/s, 24 seconds passed
... 71%, 863232 KB, 34971 KB/s, 24 seconds passed
... 72%, 864256 KB, 34998 KB/s, 24 seconds passed
... 72%, 865280 KB, 35027 KB/s, 24 seconds passed
... 72%, 866304 KB, 35039 KB/s, 24 seconds passed
... 72%, 867328 KB, 35066 KB/s, 24 seconds passed
... 72%, 868352 KB, 35064 KB/s, 24 seconds passed
... 72%, 869376 KB, 35081 KB/s, 24 seconds passed
... 72%, 870400 KB, 35097 KB/s, 24 seconds passed
... 72%, 871424 KB, 35124 KB/s, 24 seconds passed
... 72%, 872448 KB, 35153 KB/s, 24 seconds passed
... 72%, 873472 KB, 35165 KB/s, 24 seconds passed
... 72%, 874496 KB, 35193 KB/s, 24 seconds passed


... 85%, 1024000 KB, 35861 KB/s, 28 seconds passed
... 85%, 1025024 KB, 35884 KB/s, 28 seconds passed
... 85%, 1026048 KB, 35909 KB/s, 28 seconds passed
... 85%, 1027072 KB, 35891 KB/s, 28 seconds passed
... 85%, 1028096 KB, 35915 KB/s, 28 seconds passed
... 85%, 1029120 KB, 35909 KB/s, 28 seconds passed
... 85%, 1030144 KB, 35929 KB/s, 28 seconds passed
... 85%, 1031168 KB, 35953 KB/s, 28 seconds passed
... 86%, 1032192 KB, 35969 KB/s, 28 seconds passed
... 86%, 1033216 KB, 35989 KB/s, 28 seconds passed
... 86%, 1034240 KB, 35796 KB/s, 28 seconds passed
... 86%, 1035264 KB, 35819 KB/s, 28 seconds passed
... 86%, 1036288 KB, 35845 KB/s, 28 seconds passed
... 86%, 1037312 KB, 35856 KB/s, 28 seconds passed
... 86%, 1038336 KB, 35878 KB/s, 28 seconds passed
... 86%, 1039360 KB, 35807 KB/s, 29 seconds passed
... 86%, 1040384 KB, 35796 KB/s, 29 seconds passed
... 86%, 1041408 KB, 35818 KB/s, 29 seconds passed
... 86%, 1042432 KB, 35804 KB/s, 29 seconds passed
... 86%, 1043456 KB, 35827 KB/s

... 99%, 1188864 KB, 35832 KB/s, 33 seconds passed
... 99%, 1189888 KB, 35855 KB/s, 33 seconds passed
... 99%, 1190912 KB, 35841 KB/s, 33 seconds passed
... 99%, 1191936 KB, 35861 KB/s, 33 seconds passed
... 99%, 1192960 KB, 35766 KB/s, 33 seconds passed
... 99%, 1193984 KB, 35757 KB/s, 33 seconds passed
... 99%, 1195008 KB, 35777 KB/s, 33 seconds passed
... 99%, 1196032 KB, 35764 KB/s, 33 seconds passed
... 99%, 1197056 KB, 35784 KB/s, 33 seconds passed
... 99%, 1198080 KB, 35768 KB/s, 33 seconds passed
... 99%, 1199104 KB, 35725 KB/s, 33 seconds passed
... 99%, 1200128 KB, 35745 KB/s, 33 seconds passed
... 100%, 1200215 KB, 35741 KB/s, 33 seconds passed

========== Unpacking /tmp/model/public/mozilla-deepspeech-0.6.1/deepspeech-0.6.1-models.tar.gz



In [3]:
!ls -lR model

model:
total 4
drwxr-xr-x 3 dtrawins root 4096 Sep 23 13:09 public

model/public:
total 4
drwxr-xr-x 3 dtrawins root 4096 Sep 23 13:10 mozilla-deepspeech-0.6.1

model/public/mozilla-deepspeech-0.6.1:
total 4
drwxr-xr-x 2 dtrawins root 4096 Jan 10  2020 deepspeech-0.6.1-models

model/public/mozilla-deepspeech-0.6.1/deepspeech-0.6.1-models:
total 1350668
-rw-r--r-- 1 dtrawins root 945699324 Dec  3  2019 lm.binary
-rw-r--r-- 1 dtrawins root 188914896 Dec  3  2019 output_graph.pb
-rw-r--r-- 1 dtrawins root 188915850 Dec  3  2019 output_graph.pbmm
-rw-r--r-- 1 dtrawins root  47331120 Jan 10  2020 output_graph.tflite
-rw-r--r-- 1 dtrawins root  12200736 Dec  3  2019 trie


### 2. Convert our model to the Intermediate Representation (IR)

OpenVINO™ toolkit introduces its own format of graph representation and its own operation set. A graph is represented with two files: an XML file and a binary file. This representation is commonly referred to as the Intermediate Representation or IR.

`docker run` command will run a container with `omz_converter` to convert the model stored in `/tmp/model` folder and save it to `/tmp/model/converted` folder.

In [4]:
!docker run -u $(id -u) --rm -v $(pwd)/model:/tmp/model openvino/ubuntu20_dev:2022.2.0 omz_converter --name mozilla-deepspeech-0.6.1 -d /tmp/model -o /tmp/model/converted

========== Converting mozilla-deepspeech-0.6.1 to IR (FP16)
Conversion command: /usr/bin/python3.8 -- /usr/local/bin/mo --framework=tf --data_type=FP16 --output_dir=/tmp/model/converted/public/mozilla-deepspeech-0.6.1/FP16 --model_name=mozilla-deepspeech-0.6.1 --input=input_node,previous_state_h,previous_state_c --input_model=/tmp/model/public/mozilla-deepspeech-0.6.1/deepspeech-0.6.1-models/output_graph.pb '--freeze_placeholder_with_value=input_lengths->[16]' --output=logits,cudnn_lstm/rnn/multi_rnn_cell/cell_0/cudnn_compatible_lstm_cell/GatherNd,cudnn_lstm/rnn/multi_rnn_cell/cell_0/cudnn_compatible_lstm_cell/GatherNd_1 --disable_nhwc_to_nchw '--layout=input_node(NSTC),previous_state_h(NC),previous_state_c(NC)' '--input_shape=[1, 16, 19, 26],[1, 2048],[1, 2048]'

[ WARNING ]  Use of deprecated cli option --disable_nhwc_to_nchw detected. Option use in the following releases will be fatal. 
Model Optimizer arguments:
Common parameters:
	- Path to the Input Model: 	/tmp/model/public/mozi

In [5]:
!ls -lR model/converted

model/converted:
total 4
drwxr-xr-x 3 dtrawins root 4096 Sep 23 13:43 public

model/converted/public:
total 4
drwxr-xr-x 4 dtrawins root 4096 Sep 23 13:43 mozilla-deepspeech-0.6.1

model/converted/public/mozilla-deepspeech-0.6.1:
total 8
drwxr-xr-x 2 dtrawins root 4096 Sep 23 13:43 FP16
drwxr-xr-x 2 dtrawins root 4096 Sep 23 13:43 FP32

model/converted/public/mozilla-deepspeech-0.6.1/FP16:
total 92308
-rw-r--r-- 1 dtrawins root 94449844 Sep 23 13:43 mozilla-deepspeech-0.6.1.bin
-rw-r--r-- 1 dtrawins root    18764 Sep 23 13:43 mozilla-deepspeech-0.6.1.mapping
-rw-r--r-- 1 dtrawins root    47763 Sep 23 13:43 mozilla-deepspeech-0.6.1.xml

model/converted/public/mozilla-deepspeech-0.6.1/FP32:
total 184540
-rw-r--r-- 1 dtrawins root 188899568 Sep 23 13:43 mozilla-deepspeech-0.6.1.bin
-rw-r--r-- 1 dtrawins root     18764 Sep 23 13:43 mozilla-deepspeech-0.6.1.mapping
-rw-r--r-- 1 dtrawins root     39055 Sep 23 13:43 mozilla-deepspeech-0.6.1.xml


### 3. Run benchmark app on the model

Benchmark app can be used to assess the performance of the models executed by OpenVINO runtime.


In [6]:
!docker run -u $(id -u) --rm -v $(pwd)/model:/tmp/model openvino/ubuntu20_dev:2022.2.0 benchmark_app -m /tmp/model/converted/public/mozilla-deepspeech-0.6.1/FP32/mozilla-deepspeech-0.6.1.xml

['[Step 1/11] Parsing and validating input arguments',
 '[ WARNING ]  -nstreams default value is determined automatically for a device. Although the automatic selection usually provides a reasonable performance, but it still may be non-optimal for some cases, for more information look at README. ',
 '[Step 2/11] Loading OpenVINO',
 '[ WARNING ] PerformanceMode was not explicitly specified in command line. Device CPU performance hint will be set to THROUGHPUT.',
 '[ INFO ] OpenVINO:',
 '         API version............. 2022.2.0-7713-af16ea1d79a-releases/2022/2',
 '[ INFO ] Device info',
 '         CPU',
 '         openvino_intel_cpu_plugin version 2022.2',
 '         Build................... 2022.2.0-7713-af16ea1d79a-releases/2022/2',
 '',
 '[Step 3/11] Setting device configuration',
 '[ WARNING ] -nstreams default value is determined automatically for CPU device. Although the automatic selection usually provides a reasonable performance, but it still may be non-optimal for some cases,

### 3. Run a demo from an OpenVINO Model Zoo

Below will be ilustrated how to run the demos from the [OpenVINO Model Zoo repository](https://github.com/openvinotoolkit/open_model_zoo)

Start interactive session in the container with a command:

`docker run -it openvino/ubuntu20_dev:2022.2.0` 

and execute the following commmands:
```
git clone --depth=1 --recurse-submodules --shallow-submodules https://github.com/openvinotoolkit/open_model_zoo.git
cd open_model_zoo/demos/classification_demo/python
curl https://storage.openvinotoolkit.org/repositories/open_model_zoo/2022.1/models_bin/3/resnet50-binary-0001/FP32-INT1/resnet50-binary-0001.xml -o resnet50-binary-0001.xml
curl https://storage.openvinotoolkit.org/repositories/open_model_zoo/2022.1/models_bin/3/resnet50-binary-0001/FP32-INT1/resnet50-binary-0001.bin -o resnet50-binary-0001.bin
curl https://raw.githubusercontent.com/openvinotoolkit/model_server/main/demos/common/static/images/zebra.jpeg -o zebra.jpeg
python3 classification_demo.py -m resnet50-binary-0001.xml -i images/zebra.jpeg --labels ../../../data/dataset_classes/imagenet_2012.txt --no_show -nstreams 1 -r
```

The output will be like below:

In [8]:
!docker run openvino/ubuntu20_dev:2022.2.0 bash -c "git clone --depth=1 --recurse-submodules --shallow-submodules https://github.com/openvinotoolkit/open_model_zoo.git && \
   cd open_model_zoo/demos/classification_demo/python && \
   curl https://storage.openvinotoolkit.org/repositories/open_model_zoo/2022.1/models_bin/3/resnet50-binary-0001/FP32-INT1/resnet50-binary-0001.xml -o resnet50-binary-0001.xml && \
   curl https://storage.openvinotoolkit.org/repositories/open_model_zoo/2022.1/models_bin/3/resnet50-binary-0001/FP32-INT1/resnet50-binary-0001.bin -o resnet50-binary-0001.bin && \
   curl https://raw.githubusercontent.com/openvinotoolkit/model_server/main/demos/common/static/images/zebra.jpeg -o zebra.jpeg && \
   python3 classification_demo.py -m resnet50-binary-0001.xml -i zebra.jpeg --labels ../../../data/dataset_classes/imagenet_2012.txt --no_show -nstreams 1 -r "

Cloning into 'open_model_zoo'...
Submodule 'demos/thirdparty/gflags' (https://github.com/gflags/gflags.git) registered for path 'demos/thirdparty/gflags'
Cloning into '/opt/intel/openvino_2022.2.0.7713/open_model_zoo/demos/thirdparty/gflags'...
From https://github.com/gflags/gflags
 * branch            e171aa2d15ed9eb17054558e0b3a6a413bb01067 -> FETCH_HEAD
Submodule path 'demos/thirdparty/gflags': checked out 'e171aa2d15ed9eb17054558e0b3a6a413bb01067'
Submodule 'doc' (https://github.com/gflags/gflags.git) registered for path 'demos/thirdparty/gflags/doc'
Cloning into '/opt/intel/openvino_2022.2.0.7713/open_model_zoo/demos/thirdparty/gflags/doc'...
From https://github.com/gflags/gflags
 * branch            8411df715cf522606e3b1aca386ddfc0b63d34b4 -> FETCH_HEAD
Submodule path 'demos/thirdparty/gflags/doc': checked out '8411df715cf522606e3b1aca386ddfc0b63d34b4'
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   

## Part 2. Set up OpenVINO runtime container to run OpenVINO samples

The runtime image incudes a smaller number of tools and components. Most importantly it is missing openvino python development package [openvino-dev](https://pypi.org/project/openvino-dev/).

Below is an example how you could use the runtime image to build and execute the sample apps.

Start the container interactively:

`docker run -it openvino/ubuntu20_runtime:2022.2.0`

and execute the following commands:
```
./samples/cpp/build_samples.sh
/home/openvino/inference_engine_cpp_samples_build/intel64/Release/hello_query_device
```

Below is the output result:


In [9]:
!docker run openvino/ubuntu20_runtime:2022.2.0 bash -c "./samples/cpp/build_samples.sh && \
/home/openvino/inference_engine_cpp_samples_build/intel64/Release/hello_query_device"


Setting environment variables for building samples...
[setupvars.sh] OpenVINO environment initialized
-- The C compiler identification is GNU 9.4.0
-- The CXX compiler identification is GNU 9.4.0
-- Check for working C compiler: /usr/bin/cc
-- Check for working C compiler: /usr/bin/cc -- works
-- Detecting C compiler ABI info
-- Detecting C compiler ABI info - done
-- Detecting C compile features
-- Detecting C compile features - done
-- Check for working CXX compiler: /usr/bin/c++
-- Check for working CXX compiler: /usr/bin/c++ -- works
-- Detecting CXX compiler ABI info
-- Detecting CXX compiler ABI info - done
-- Detecting CXX compile features
-- Detecting CXX compile features - done
-- Looking for C++ include unistd.h
-- Looking for C++ include unistd.h - found
-- Looking for C++ include stdint.h
-- Looking for C++ include stdint.h - found
-- Looking for C++ include sys/types.h
-- Looking for C++ include sys/types.h - found
-- Looking for C++ include fnmatch.h
-- Looking for C++ i


Build completed, you can find binaries for all samples in the /home/openvino/inference_engine_cpp_samples_build/intel64/Release subfolder.

[ INFO ] OpenVINO Runtime version ......... 2022.2.0
[ INFO ] Build ........... 2022.2.0-7713-af16ea1d79a-releases/2022/2
[ INFO ] 
[E:] [BSL] found 0 ioexpander device
E: [xLinkUsb] [    599966] [hello_query_dev] usb_find_device_with_bcd:266	Library has not been initialized when loaded
[ INFO ] Available devices: 
[ INFO ] CPU
[ INFO ] 	SUPPORTED_PROPERTIES: 
[ INFO ] 		Immutable: AVAILABLE_DEVICES : ""
[ INFO ] 		Immutable: RANGE_FOR_ASYNC_INFER_REQUESTS : 1 1 1
[ INFO ] 		Immutable: RANGE_FOR_STREAMS : 1 96
[ INFO ] 		Immutable: FULL_DEVICE_NAME : Intel(R) Xeon(R) Gold 6252 CPU @ 2.10GHz        
[ INFO ] 		Immutable: OPTIMIZATION_CAPABILITIES : WINOGRAD FP32 FP16 INT8 BIN EXPORT_IMPORT
[ INFO ] 		Immutable: CACHE_DIR : ""
[ INFO ] 		Mutable: NUM_STREAMS : 1
[ INFO ] 		Mutable: AFFINITY : CORE
[ INFO ] 		Mutable: INFERENCE_NUM_THREADS : 0
[ INFO

## Part 3. Using AI accelerators

Of course, you can run a container and inference on different Intel devices like iGPU (Intel® Processor Graphics), NCS2(Intel® Neural Compute Stick 2) or HDDL (Intel® Vision Accelerator Design with Intel® Movidius™ VPUs). 

To enable GPU access, add `--device /dev/dri` to `docker run`. You might also need to add to the container security context a group with permission to the GPU character device.

You can find more information about how to run a container with access to a specific device on our [Docker CI GitHub repo](https://github.com/openvinotoolkit/docker_ci/blob/master/get-started.md#run-a-container)

In [10]:
!docker run -it --device /dev/dri --group-add=$(stat -c "%g" /dev/dri/render* ) openvino/ubuntu20_dev:2022.2.0 ./samples/cpp/samples_bin/hello_query_device

[ INFO ] OpenVINO Runtime version ......... 2022.2.0
[ INFO ] Build ........... 2022.2.0-7713-af16ea1d79a-releases/2022/2
[ INFO ]
[E:] [BSL] found 0 ioexpander device
E: [xLinkUsb] [    112159] [hello_query_dev] usb_find_device_with_bcd:266       Library has not been initialized when loaded
[ INFO ] Available devices:
[ INFO ] CPU
[ INFO ]        SUPPORTED_PROPERTIES:
[ INFO ]                Immutable: AVAILABLE_DEVICES : ""
[ INFO ]                Immutable: RANGE_FOR_ASYNC_INFER_REQUESTS : 1 1 1
[ INFO ]                Immutable: RANGE_FOR_STREAMS : 1 8
[ INFO ]                Immutable: FULL_DEVICE_NAME : Intel(R) Core(TM) i3-10100 CPU @ 3.60GHz
[ INFO ]                Immutable: OPTIMIZATION_CAPABILITIES : FP32 FP16 INT8 BIN EXPORT_IMPORT
[ INFO ]                Immutable: CACHE_DIR : ""
[ INFO ]                Mutable: NUM_STREAMS : 1
[ INFO ]                Mutable: AFFINITY : CORE
[ INFO ]                Mutable: INFERENCE_NUM_THREADS : 0
[ INFO ]                Mutable: PERF_C

You can find more information about how to run a container with access to a specific device on our [Docker CI GitHub repo](https://github.com/openvinotoolkit/docker_ci/blob/master/get-started.md#run-a-container)

## Summary

In this article, we briefly introduced the speech recognition demo using OpenVINO Docker images. Of course, there is much more to try. We hope that this article has motivated you to try it yourself and maybe continue to explore all the possibilities of OpenVINO Docker images.



## References



*   [Available OpenVINO Docker images](https://github.com/openvinotoolkit/docker_ci#prebuilt-images)
*   [Docker CI framework for Intel® Distribution of OpenVINO™ toolkit](https://github.com/openvinotoolkit/docker_ci). The Framework can generate a Dockerfile, build, test, and deploy an image with the Intel® Distribution of OpenVINO™ toolkit. You can reuse available Dockerfiles, add your layer and customize the image of OpenVINO™ for your needs.

